In [1]:
!pip install mtcnn
!pip install kaggle

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("elin75/localized-audio-visual-deepfake-dataset-lav-df")

print("Path to dataset files:", path)

Path to dataset files: /teamspace/studios/this_studio/.cache/kagglehub/datasets/elin75/localized-audio-visual-deepfake-dataset-lav-df/versions/1


In [5]:
!pip install imblearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [imblearn]1/3 [imbalanced-learn]


In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import add, BatchNormalization, GlobalAveragePooling2D, LSTM, Dense, Dropout, Input, Masking
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input  # Or any other preprocessing based on your model
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import RocCurveDisplay

2025-06-12 05:01:39.722925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749704499.909647    2117 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749704499.966063    2117 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749704500.526495    2117 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749704500.526923    2117 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749704500.526934    2117 computation_placer.cc:177] computation placer alr

In [4]:
Datagen=ImageDataGenerator(rescale=1./255)

In [5]:
import os
import cv2
import json
from tqdm import tqdm

# Constants
VIDEO_ROOT = "/teamspace/studios/this_studio/.cache/kagglehub/datasets/elin75/localized-audio-visual-deepfake-dataset-lav-df/versions/1/LAV-DF"
FRAME_OUTPUT_DIR = "LAV-DF-Frames"
METADATA_PATH = os.path.join(VIDEO_ROOT, "metadata.min.json")
FRAME_SIZE = (256, 256)
MAX_FRAMES = 30

# Load metadata
with open(METADATA_PATH, "r") as f:
    metadata = json.load(f)

# Select 50 test, 50 train, 50 dev
test_entries, train_entries, dev_entries = [], [], []

for entry in metadata:
    if entry["split"] == "test" and len(test_entries) < 800:
        test_entries.append(entry)
    elif entry["split"] == "train" and len(train_entries) < 2000:
        train_entries.append(entry)
    elif entry["split"] == "dev" and len(dev_entries) < 1000:
        dev_entries.append(entry)

    if len(test_entries) == 800 and len(train_entries) == 2000 and len(dev_entries) == 1000:
        break

selected_entries = test_entries + train_entries + dev_entries


# Frame extractor
def extract_and_save_frames(video_path, label, subset, video_id):
    cap = cv2.VideoCapture(video_path)
    count = 0
    output_dir = os.path.join(FRAME_OUTPUT_DIR, subset, label, video_id)
    os.makedirs(output_dir, exist_ok=True)

    while count < MAX_FRAMES:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, FRAME_SIZE)
        cv2.imwrite(os.path.join(output_dir, f"{count}.jpg"), frame)
        count += 1
    cap.release()

# Process selected entries
for entry in tqdm(selected_entries, desc="Extracting frames"):
    file_path = entry["file"]  # e.g., 'train/000001.mp4'
    subset = entry["split"]
    video_id = os.path.splitext(os.path.basename(file_path))[0]
    label = "fake" if entry["modify_video"] else "real"
    video_path = os.path.join(VIDEO_ROOT, file_path)

    extract_and_save_frames(video_path, label, subset, video_id)


Extracting frames: 100%|██████████| 3800/3800 [05:48<00:00, 10.91it/s]


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    "LAV-DF-Frames/train",
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

valid_gen = datagen.flow_from_directory(
    "LAV-DF-Frames/dev",
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary'
)

Found 60000 images belonging to 2 classes.
Found 30000 images belonging to 2 classes.


In [14]:
model = InceptionV3(
    include_top = False,
    weights = "imagenet",
    input_shape = (256,256,3)
)

for layer in model.layers:
  layer.trainable = False

# Get the output of the InceptionV3 model
x = model.output

# Add new layers on top of the InceptionV3 output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling
x = Dense(128, activation='relu', kernel_regularizer=l2(0.05))(x)  # Dense layer with 128 units
x = BatchNormalization()(x)  # Batch Normalization
x = Dropout(0.5)(x)  # Dropout layer with 50% dropout rate
x = Dense(64, activation='relu', kernel_regularizer=l2(0.05))(x)  # Dense layer with 64 units
x = BatchNormalization()(x)  # Batch Normalization
x = Dropout(0.2)(x)  # Dropout layer with 50% dropout rate
x = Dense(1, activation='sigmoid')(x)  # Output layer with sigmoid activation

# Create a new model with the specified inputs and outputs
model1 = Model(inputs=model.input, outputs=x)

model1.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_94 (Conv2D)  │ (None, 127, 127,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 127, 127,  │         96 │ conv2d_94[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_94       │ (None, 127, 127,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_95 (Conv2D)  │ (None, 125, 125,  │      9,216 │ activation_94[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 125, 125,  │         96 │ conv2d_95[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_95       │ (None, 125, 125,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_96 (Conv2D)  │ (None, 125, 125,  │     18,432 │ activation_95[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 125, 125,  │        192 │ conv2d_96[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_96       │ (None, 125, 125,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 62, 62,    │          0 │ activation_96[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_97 (Conv2D)  │ (None, 62, 62,    │      5,120 │ max_pooling2d_4[… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 62, 62,    │        240 │ conv2d_97[0][0]   │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_97       │ (None, 62, 62,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_98 (Conv2D)  │ (None, 60, 60,    │    138,240 │ activation_97[0]… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 60,    │        576 │ conv2d_98[0][0]   │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_98       │ (None, 60, 60,    │          0 │ batch_normalizat

 Total params: 22,074,145 (84.21 MB)

 Trainable params: 270,977 (1.03 MB)

 Non-trainable params: 21,803,168 (83.17 MB)

In [1]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# import precision, Recall
from tensorflow.keras.metrics import Precision, Recall
# from tensorflow_addons.metrics import F1Score  # If using TensorFlow Addons

# Compile the model
model1.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss="binary_crossentropy",
    metrics=["accuracy", Precision(), Recall()]
)
# Callbacks
earlystopping = EarlyStopping(
    monitor="val_loss",
    verbose=1,
    patience = 200,
    restore_best_weights=True
)

modelcheckpoint = ModelCheckpoint(
    "InceptionV3ModelUpdated.h5",
    monitor="val_loss",
    verbose=1,
    save_best_only=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    verbose=1,
    min_lr=1e-6,
    patience = 1
)

# Training the model
history = model1.fit(
    train_gen,
    epochs=1000,
    validation_data=valid_gen,
    verbose=1,
    callbacks=[earlystopping, modelcheckpoint, reduce_lr]
)

2025-06-12 01:52:40.001814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749693160.197150    2293 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749693160.280010    2293 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749693160.774506    2293 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749693160.774970    2293 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749693160.774979    2293 computation_placer.cc:177] computation placer alr

NameError: name 'model1' is not defined

In [7]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

# Load the saved best model
model1 = load_model("InceptionV3ModelUpdated.h5")

# Recompile the model (important after loading)
model1.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss="binary_crossentropy",
    metrics=["accuracy", Precision(), Recall()]
)

# Callbacks
earlystopping = EarlyStopping(
    monitor="val_loss",
    verbose=1,
    patience=200,
    restore_best_weights=True
)

modelcheckpoint = ModelCheckpoint(
    "InceptionV3ModelUpdated.h5",  # overwrite with the better one if it appears
    monitor="val_loss",
    verbose=1,
    save_best_only=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    verbose=1,
    min_lr=1e-6,
    patience=1
)

# Resume training from epoch 20
initial_epoch = 20
total_epochs = 1000

history = model1.fit(
    train_gen,
    epochs=total_epochs,
    initial_epoch=initial_epoch,
    validation_data=valid_gen,
    verbose=1,
    callbacks=[earlystopping, modelcheckpoint, reduce_lr]
)


I0000 00:00:1749704867.008236    2117 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13775 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 21/1000


I0000 00:00:1749704880.176914   35549 service.cc:152] XLA service 0x7c0f20004ae0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749704880.177564   35549 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-06-12 05:08:00.603507: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1749704883.381003   35549 cuda_dnn.cc:529] Loaded cuDNN version 90701


   1/1875 ━━━━━━━━━━━━━━━━━━━━ 11:34:38 22s/step - accuracy: 0.5938 - loss: 0.7103 - precision: 0.5714 - recall: 0.5333

I0000 00:00:1749704893.019087   35549 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.5487 - loss: 0.7068 - precision: 0.5617 - recall: 0.5060
Epoch 21: val_loss improved from inf to 0.71862, saving model to InceptionV3ModelUpdated.h5


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 260s 127ms/step - accuracy: 0.5487 - loss: 0.7068 - precision: 0.5617 - recall: 0.5060 - val_accuracy: 0.5145 - val_loss: 0.7186 - val_precision: 0.5883 - val_recall: 0.1161 - learning_rate: 5.0000e-04
Epoch 22/1000
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.5536 - loss: 0.7045 - precision: 0.5670 - recall: 0.5033
Epoch 22: val_loss did not improve from 0.71862

Epoch 22: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 230s 123ms/step - accuracy: 0.5536 - loss: 0.7045 - precision: 0.5670 - recall: 0.5033 - val_accuracy: 0.4988 - val_loss: 0.7320 - val_precision: 0.6687 - val_recall: 0.0072 - learning_rate: 5.0000e-04
Epoch 23/1000
 126/1875 ━━━━━━━━━━━━━━━━━━━━ 2:25 83ms/step - accuracy: 0.5492 - loss: 0.7037 - precision: 0.5511 - recall: 0.4875

KeyboardInterrupt: 

In [ ]:
test_gen=Datagen.flow_from_directory(
    'LAV-DF-Frames/test',
    target_size=(256,256),
    batch_size=64,
    class_mode='binary',
    shuffle=True
)

In [ ]:
loss, accuracy, precision, recall = model.evaluate(testgen, verbose=1)